In [1]:
# install dependencies: 
!pip install pyyaml==5.1 'pycocotools>=2.0.1'
#!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
#!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 8.7MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=d6d19cb2bd7be63b06f618b8c105ec8b0533f18be3d60d437da3f5a9ca3b8188
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.7.0+cu101 True


In [2]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
     |████████████████████████████████| 6.6MB 7.1MB/s 
     |████████████████████████████████| 2.2MB 14.3MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.2.post20201218-cp36-none-any.whl size=40701 sha256=a109519f1cec3a92071067654bd18817453780621d94458fbae28adcce8ad228
  Stored in directory: /root/.cache/pip/wheels/b5/83/3f/e74a72d264351b5ada512da845a5c6abb81a34f6333c3a54ce
  Created wheel for iopath: filename=iopath-0.1.2-cp36-none-any.whl size=10508 sha256=c4111dbefb0dcdff7cd50a08873ed0bc1a5e4949c6643836bb2b045f3b2bb647
  Stored in directory: /root/.cache/pip/wheels/9e/01/e4/1b68f5a2a6b9450ea4246d91840a77e1169f7d4722d76bbc47
Successfully built fvcore iopath
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.

In [1]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
import time
from detectron2.utils.visualizer import ColorMode

import matplotlib.pyplot as plt
import matplotlib as mpl
import logging
import threading
import time
import math
from detectron2.data.datasets import register_coco_instances

In [3]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


To save a model, run the following

In [ ]:
#To save a model, run:
# !cp 'output/model_final.pth' 'drive/My Drive/CIS581/FinalProject/Ruleset/outputfile.pth'
#Do this directly after the training stage

For Loading a pretrained Model do the following (If takes more than 1 minute to run, check if you have the GPU enabled):

In [4]:
#To load a model run:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 14
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
cfg.MODEL.WEIGHTS ="/content/drive/My Drive/CIS581/FinalProject/Ruleset/Highres.pth"# path to the model we trained
predictor = DefaultPredictor(cfg)


Homography

Running Live Video Detections

In [42]:
def threading_funct(frame):
  outputs = predictor(frame) 
  data=outputs["instances"].to("cpu")
  output_mask=data.pred_masks.numpy().astype(int)
  output_classes=data.pred_classes.numpy()
  count=output_classes.shape
  mask=output_mask*output_classes[:,np.newaxis, np.newaxis]
  #print("mask shape=",mask.shape)

  #print("max value=",max(mask))
  #mask=np.where(mask==3,18,mask)
  #print("mask shape=",mask.shape)
  #newimage1=np.amax(mask,axis=0)


  mask=np.where(mask==2,mask,0)
  image_seg=np.amax (mask,axis=0)
  newimage1=np.where(image_seg==2,1,0)
  road=newimage1

  xmin=road.shape[1]
  xmax=0
  ymin=road.shape[0]
  ymax=0
  for i in range(road.shape[0]):
    for j in range(road.shape[1]):
      if road[i,j]==1:
        if i<ymin:
          ymin=i
        if j<xmin:
          xmin=j
        if i>ymax:
          ymax=i
        if j>xmax:
          xmax=j  
  xmaxtop=0
  xmintop=road.shape[1]
  for i in range(ymin,ymin+15):
    for j in range(xmin,xmax):
      if road[i,j]==1:
        if j>xmaxtop:
            xmaxtop=j
        if j<xmintop:
            xmintop=j
  newimage1=(road*25).astype(np.uint8)
  newimage1=cv2.cvtColor(newimage1, cv2.COLOR_RGB2BGR)
  newimage1=cv2.applyColorMap(newimage1,cv2.COLORMAP_JET) 
  cv2.circle(newimage1,(xmax,ymax), 10, (0,255,0), -1)
  cv2.circle(newimage1,(xmin,ymax), 10, (0,255,0), -1)
  cv2.circle(newimage1,(xmintop,ymin), 10, (0,255,0), -1)
  cv2.circle(newimage1,(xmaxtop,ymin), 10, (0,255,0), -1)

  #cv2.circle(newimage1,(0,ymin), 10, (255,105,0), -1)
  #cv2.circle(newimage1,(480,ymin), 10, (255,105,0), -1)

  pts1=np.float32([[xmax,ymax],[xmin,ymax],[xmaxtop,ymin],[xmintop,ymin]])
  
  #pts2=np.float32([[abs(xmax-xmaxtop)+min(xmax,xmaxtop),ymax],[abs(xmin-xmintop),ymax],[abs(xmax-xmaxtop)+min(xmax,xmaxtop)+75,ymin],[abs(xmin-xmintop)-75,ymin]])
  pts2=np.float32([[300,ymax],[180,ymax],[480,ymin],[0,ymin]])

  M = cv2.getPerspectiveTransform(pts1,pts2)
  dst = cv2.warpPerspective(frame,M,(road.shape[1],road.shape[0]))
  #cv2_imshow(dst)

  ymin=max(ymin-5  ,0)   
  #cv2_imshow(road*25)
  #global value
  #value=outputs["instances"].to("cpu").pred_boxes.tensor.numpy().astype(float)[3]
  #global start
  #start=(int((value[2]+value[0])/2),int((value[3]+value[1])/2))  
  global endpoint
  endpoint=(int(line_length*math.cos(np.deg2rad(arrow_angle)))+start[0],int(line_length*math.sin(np.deg2rad(arrow_angle)))+start[1])
  #Checking for Intersection
  line_y=100

  x_new=(math.cos(np.deg2rad(arrow_angle)))*10+start[0]
  global y_new, y1
  y_new=-(math.sin(np.deg2rad(arrow_angle))*10)+start[1]
  x1=start[0]
  y1=start[1]
  if (x1-x_new==0):
    x_new+=0.0000001
  if (y1-y_new==0):
    y_new+=0.000001  
  slope=((y1-y_new)/(x1-x_new))
  b=y1-(slope*x1)
  global intersect_loc_x
  intersect_loc_x=(100-b)/slope#(slope*line_y)+b
 
  global newimage
  newimage=newimage1
  global ready
  ready=True

  global newimageframe
  newimageframe=frame
  global transform
  transform=dst



print("set video fps before running this code")
print("Code may fail on first run, if this happens try running again.")
fps=29.97
print("fps=",fps)


#video=cv2.VideoCapture('480_360vid.mp4')
video=cv2.VideoCapture('/content/drive/My Drive/CIS581/FinalProject/videos_raw/twolane2020-12-19.mp4')

frame_width = int(video.get(3)*3) 
frame_height = int(video.get(4)) 
   
size = (frame_width, frame_height) 
output=cv2.VideoWriter("/content/drive/My Drive/CIS581/FinalProject/GoodHomographyExample.mp4",  
                         cv2.VideoWriter_fourcc(*'Mp4V'), 
                         fps, size)
ready=True
counter=0
starttime=time.time()


arrow_angle=0
start=(0,0)
line_length=30
endpoint=(0,0)
intersect_loc_x=10000
minbound=0.3*(frame_width/2)
maxbound=(frame_width/2)-(0.3*(frame_width/2))

while(True):
  t=time.time()
  
  ret,frame=video.read()
 
  if ret==True:
    if ready==True:
      if counter==0:
        outputim=frame
      ready=False
      x=threading.Thread(target=threading_funct, args=(frame,) )
      x.start()   
    else:
      if counter<0:
        outputim=frame
        newimageframe=frame
      else:  
        outputim=newimage#(newimage*25).astype(np.uint8)
        #outputim=cv2.cvtColor(outputim, cv2.COLOR_RGB2BGR)
        #outputim=cv2.applyColorMap(outputim,cv2.COLORMAP_JET)  
    
    if counter==0:
      outputs = predictor(frame)
      newimageframe=frame
    arrow_angle+=0.7
    if arrow_angle>=360:
      arrow_angle=0
    #cv2.rectangle(frame, (int(value[0]),int(value[1])), (int(value[2]),int(value[3])), (0,100,100), 3) 
    #cv2.rectangle(frame, (math.floor(minbound),math.floor(frame_height-100)), (math.ceil(maxbound),math.ceil(frame_height)), (255,0,0), 7) 
    
   # if intersect_loc_x > minbound and intersect_loc_x < maxbound and y_new<y1:
     # cv2.arrowedLine(frame,start,endpoint,(0,0,255),2)
   # else:
    #  cv2.arrowedLine(frame,start,endpoint,(0,255,0),2)
  
    #outputim1=cv2.addWeighted(newimageframe,0.5,outputim,0.5,0)
    outputimage=cv2.hconcat([frame,outputim,transform])
    #cv2_imshow(frame)
    #cv2_imshow(outputim)
    output.write(outputimage)
    counter=1
    timediff=time.time()-t

    if timediff < (1/fps):
     time.sleep((1.0/fps)-timediff)

  else:
    break
  

video.release()
output.release()
print("total Time=",time.time()-starttime)
print("Must check length of original video, if length is longer than")
print("total time, the program did not run in real time.")

set video fps before running this code
Code may fail on first run, if this happens try running again.
fps= 29.97
total Time= 42.37425446510315
Must check length of original video, if length is longer than
total time, the program did not run in real time.


In [28]:
xmax

NameError: ignored

In [29]:
xmaxtop

NameError: ignored